<a href="https://colab.research.google.com/github/tali9943/DEEP-LEARNING-FOR-NATURAL-LANGUAGE-PROCESSING/blob/main/First_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dataset


In [47]:
# load google drive to see the files in google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
train_csv_path = '/content/drive/MyDrive/Progetto-Vascon/Dataset/train.csv'
test_csv_path = '/content/drive/MyDrive/Progetto-Vascon/Dataset/test.csv'

In [49]:
#function to load a csv file from a drive folder
import pandas as pd

def load_csv(path_csv):
  path_file_csv = path_csv
  df = pd.read_csv(path_file_csv)

  return df

In [50]:
test_df = load_csv(test_csv_path)
test_df

In [ ]:
train_df = load_csv(train_csv_path)
train_df

In [ ]:
import pandas
pandas.set_option('display.max_rows', None)

"""
Function that creates a dataframe which takes the input number of rows from an input dataframe
"""
def take_first_rows_dataframe(train_df, n_rows):
    return train_df.head(n_rows)


# Showing the final training set
train_df = take_first_rows_dataframe(train_df, 500)
train_df

## Stemming and Lemming

In [ ]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Scarica le stop words se non le hai già
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


import string

def perform_stemming(sentence):
    ps = PorterStemmer()
    words = word_tokenize(sentence)
    stemmed_words = [ps.stem(word) for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)

def perform_lemmatization(sentence):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(sentence)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(lemmatized_words)

def preprocess_text(text):
    # Rimuovi la punteggiatura
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Converti tutto in minuscolo
    text = text.lower()
    return text

# Applica la funzione di preprocessing alla colonna 'comment_text' di train_df
train_df['comment_text'] = train_df['comment_text'].apply(preprocess_text).apply(perform_stemming).apply(perform_lemmatization)



In [ ]:
train_df

# Prove BERT

sistemare le frasi con stemming e lemming prima di passarlo a bert!

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn as nn

# Carica il tokenizer di BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Carica il modello preaddestrato di BERT per classificazione di sequenze
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


# Aggiungi uno strato di classificazione sopra il modello preaddestrato
num_classes = 6  # Numero di classi del tuo task
model.classifier = nn.Linear(model.config.hidden_size, num_classes)

# Opzionale: fissa i pesi del modello preaddestrato
for param in model.base_model.parameters():
    param.requires_grad = False


# Prova per riallenare bert con i nostri dati

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Carica i dati e converti le etichette in tensori
# Assumi che train_df contenga le colonne 'comment_text' e le 6 classi binarie
texts = train_df['comment_text'].values
labels = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# Dividi i dati in set di addestramento, set di validazione e set di test
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)

# Carica il tokenizer e il modello preaddestrato di BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Tokenizza i testi e converte in tensori
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')

# Creazione di tensori per le etichette
train_labels = torch.tensor(train_labels, dtype=torch.float32)
val_labels = torch.tensor(val_labels, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.float32)

# Creazione di DataLoader per gestire i dati durante l'addestramento e la validazione
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

# Creazione di DataLoader per gestire i dati durante l'addestramento e la validazione
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Imposta i parametri di ottimizzazione e il dispositivo di addestramento
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Addestra il modello con validazione
num_epochs = 3
for epoch in range(num_epochs):
    # Fase di addestramento
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

    # Fase di validazione
    model.eval()
    val_predictions = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            val_predictions.extend(torch.sigmoid(logits).cpu().numpy())

    # Calcola l'accuratezza sulla fase di validazione
    val_predictions = (np.array(val_predictions) > 0.5).astype(int)
    val_accuracy = accuracy_score(val_labels.numpy(), val_predictions)
    print(f'Epoch {epoch + 1}/{num_epochs} - Validation Accuracy: {val_accuracy}')

# Addestra il modello sul set di addestramento completo (senza validazione)
model.train()
for batch in tqdm(train_loader, desc=f'Final Training Epoch'):
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()

    optimizer.step()

# Valuta il modello sul set di test
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating - Test'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        test_predictions.extend(torch.sigmoid(logits).cpu().numpy())

# Calcola l'accuratezza sul set di test
test_predictions = (np.array(test_predictions) > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels.numpy(), test_predictions)
print(f'Test Accuracy: {test_accuracy}')

# Stampa il report di classificazione sul set di test
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(classification_report(test_labels.numpy(), test_predictions, target_names=class_names))

